In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/dataset/


Control  MS


In [ ]:
pip install --upgrade torchvision


Training 1000 MS and Control data using VGG model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms # Corrected the import statement
from torch.utils.data import DataLoader, random_split

# Define transformations for training and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Set up dataset paths (assume images are organized into 'Control' and 'MS' subfolders)
dataset_path = '/content/drive/MyDrive/dataset'

dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms['train'])


# Split dataset (65% train, 15% validation, 20% test)
train_size = int(0.65 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)


model = models.vgg16(pretrained=True)

# VGG16 has a classifier attribute instead of fc
num_features = model.classifier[6].in_features  # Get the input features of the last layer in the classifier
model.classifier[6] = nn.Linear(num_features, 2)  # Modify the last layer for binary classification

# Set up training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training and validation loop
num_epochs = 10
best_val_accuracy = 0.0
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)
        total_preds += labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_preds.double() / len(train_loader.dataset)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            val_total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct.double() / len(val_loader.dataset)

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_resnet18_model.pth')

    # Print epoch results
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# Testing phase
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels.data)
        test_total += labels.size(0)

test_accuracy = test_correct.double() / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save final model after training
torch.save(model.state_dict(), 'final_models.vgg16_model.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 0.5656, Train Acc: 0.7053, Val Loss: 0.4138, Val Acc: 0.7949
Epoch 2/10, Train Loss: 0.4198, Train Acc: 0.8095, Val Loss: 0.2723, Val Acc: 0.9077
Epoch 3/10, Train Loss: 0.2671, Train Acc: 0.8817, Val Loss: 0.3005, Val Acc: 0.8410
Epoch 4/10, Train Loss: 0.2587, Train Acc: 0.8970, Val Loss: 0.4472, Val Acc: 0.7744
Epoch 5/10, Train Loss: 0.1433, Train Acc: 0.9420, Val Loss: 0.2044, Val Acc: 0.9026
Epoch 6/10, Train Loss: 0.1745, Train Acc: 0.9456, Val Loss: 0.1749, Val Acc: 0.9487
Epoch 7/10, Train Loss: 0.0988, Train Acc: 0.9621, Val Loss: 0.1374, Val Acc: 0.9487
Epoch 8/10, Train Loss: 0.0779, Train Acc: 0.9751, Val Loss: 0.2946, Val Acc: 0.8923
Epoch 9/10, Train Loss: 0.0918, Train Acc: 0.9598, Val Loss: 0.1568, Val Acc: 0.9436
Epoch 10/10, Train Loss: 0.0304, Train Acc: 0.9941, Val Loss: 0.1951, Val Acc: 0.9333
Test Accuracy: 0.9500


Model using ResNet 18

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
#from torchvision import datasets, models, transforams
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

# Define transformations for training and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Set up dataset paths (assume images are organized into 'Control' and 'MS' subfolders)
dataset_path = '/content/drive/MyDrive/dataset'

dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms['train'])


# Split dataset (65% train, 15% validation, 20% test)
train_size = int(0.65 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Load pretrained ResNet-18 model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Modify output layer for binary classification (Healthy vs MS)

# Set up training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training and validation loop
num_epochs = 10
best_val_accuracy = 0.0
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)
        total_preds += labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_preds.double() / len(train_loader.dataset)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            val_total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct.double() / len(val_loader.dataset)

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_resnet18_model.pth')

    # Print epoch results
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# Testing phase
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels.data)
        test_total += labels.size(0)

test_accuracy = test_correct.double() / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save final model after training
torch.save(model.state_dict(), 'final_resnet18_model.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 188MB/s]


Epoch 1/10, Train Loss: 0.4875, Train Acc: 0.7586, Val Loss: 0.3867, Val Acc: 0.8205
Epoch 2/10, Train Loss: 0.2939, Train Acc: 0.8769, Val Loss: 0.3341, Val Acc: 0.8410
Epoch 3/10, Train Loss: 0.2130, Train Acc: 0.9148, Val Loss: 0.3265, Val Acc: 0.8410
Epoch 4/10, Train Loss: 0.1425, Train Acc: 0.9467, Val Loss: 0.2726, Val Acc: 0.8821
Epoch 5/10, Train Loss: 0.1201, Train Acc: 0.9633, Val Loss: 0.1959, Val Acc: 0.9077
Epoch 6/10, Train Loss: 0.0792, Train Acc: 0.9763, Val Loss: 0.3527, Val Acc: 0.8615
Epoch 7/10, Train Loss: 0.1177, Train Acc: 0.9538, Val Loss: 0.4426, Val Acc: 0.8256
Epoch 8/10, Train Loss: 0.0678, Train Acc: 0.9834, Val Loss: 0.1329, Val Acc: 0.9487
Epoch 9/10, Train Loss: 0.0895, Train Acc: 0.9621, Val Loss: 0.8874, Val Acc: 0.6718
Epoch 10/10, Train Loss: 0.0701, Train Acc: 0.9728, Val Loss: 0.3260, Val Acc: 0.8667
Test Accuracy: 0.8885


Model using ResNet-50

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforams
from torch.utils.data import DataLoader, random_split

# Define transformations for training and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Set up dataset paths (assume images are organized into 'Control' and 'MS' subfolders)
dataset_path = '/content/drive/MyDrive/dataset'

dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms['train'])


# Split dataset (65% train, 15% validation, 20% test)
train_size = int(0.65 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Load pretrained ResNet-50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Modify output layer for binary classification (Healthy vs MS)

# Set up training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training and validation loop
num_epochs = 10
best_val_accuracy = 0.0
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)
        total_preds += labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_preds.double() / len(train_loader.dataset)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            val_total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct.double() / len(val_loader.dataset)

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_resnet18_model.pth')

    # Print epoch results
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# Testing phase
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels.data)
        test_total += labels.size(0)

test_accuracy = test_correct.double() / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save final model after training
torch.save(model.state_dict(), 'final_resnet18_model.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 119MB/s]


Epoch 1/10, Train Loss: 0.5450, Train Acc: 0.6935, Val Loss: 0.4773, Val Acc: 0.7538
Epoch 2/10, Train Loss: 0.2904, Train Acc: 0.8710, Val Loss: 0.4080, Val Acc: 0.8205
Epoch 3/10, Train Loss: 0.1526, Train Acc: 0.9467, Val Loss: 0.4640, Val Acc: 0.7744
Epoch 4/10, Train Loss: 0.1122, Train Acc: 0.9574, Val Loss: 0.8215, Val Acc: 0.6872
Epoch 5/10, Train Loss: 0.0948, Train Acc: 0.9669, Val Loss: 0.3230, Val Acc: 0.8821
Epoch 6/10, Train Loss: 0.1715, Train Acc: 0.9349, Val Loss: 0.6044, Val Acc: 0.7282
Epoch 7/10, Train Loss: 0.1044, Train Acc: 0.9633, Val Loss: 0.9403, Val Acc: 0.6923
Epoch 8/10, Train Loss: 0.0668, Train Acc: 0.9740, Val Loss: 0.2108, Val Acc: 0.9026
Epoch 9/10, Train Loss: 0.0180, Train Acc: 0.9953, Val Loss: 0.1913, Val Acc: 0.9333
Epoch 10/10, Train Loss: 0.0825, Train Acc: 0.9775, Val Loss: 0.2458, Val Acc: 0.9231
Test Accuracy: 0.8923


In [ ]:
# Save the model
torch.save(model.state_dict(), 'model.pth')


In [ ]:
models.vgg16